In [ ]:
import numpy as np
from numba import njit
from scipy.linalg import cholesky
import anDiffReg as an

In [ ]:
# exemplary data: simulated fractional Brownian motion

n = 10**4 # number of trajectories
alpha, D = 0.8, 1 # FBM parameters: Hurst index H and diffusivity D
ln = 100 # trajectory length
dt = 1 # time interval
ts = dt * np.arange(1, ln+1)


def K(s, t): # FBM covariance
    return D * (t**alpha + s**alpha - np.abs(s - t)**alpha)

S = np.array([[K(s, t) for t in ts] for s in ts])
A = cholesky(S, lower=False)
xi = np.random.randn(len(ts), n)
X = A.T @ xi
xi = np.random.randn(len(ts), n) 
Y = A.T @ xi

In [ ]:
# alternatively, load data from the file

import pandas as pd
df = pd.read_csv('exemplaryData/2D FBM.csv')
X, Y = df.values[:,::2], df.values[:,1::2]

ln, n = X.shape
dt = 1 
ts = dt * np.arange(1, ln+1)
alpha = 0.8 # used only optionally for initial estimate

In [ ]:
# TA-MSD analysis

msd = an.tamsd(np.dstack((X,Y))) # TA-MSD of 2D traj
ols, covOLS = an.fit_ols(msd, 2, dt)

In [ ]:
np.mean(ols, axis = 1)  # average estimates

In [ ]:

np.cov(ols) # covariance of the estimates, compare with predicted covOLS

In [ ]:
gls, covGLS = an.fit_gls(msd, 2, dt, np.full(10**4,0.8)) 

In [ ]:
np.mean(gls, axis = 1)

In [ ]:
np.cov(gls)

In [ ]:
# covariance is computed for each trajectory, could be slow if there are many

glsP, covGLSP = an.fit_gls(msd[:,:100], 2, dt, np.full(10**4,0.8), precompute = False)

In [ ]:
## trajectories with noise

D, sigma = 5, 1

X2 = np.sqrt(D)*X + sigma*np.random.randn(len(ts), n)
msd2 = an.tamsd(X2)

# initial estimates of bot alpha and D must be used, sigma must be known 
gls2, covGLS2 = an.fit_gls(msd2, 1, 1, np.full(10**4,alpha), np.full(10**4,D), sigma)

In [ ]:
np.mean(gls2, axis=1), np.cov(gls2)